In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('GSAF5.csv', sep=',',engine='python', encoding='latin1')
#df

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [4]:
df.shape

(5992, 24)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

In [6]:
for col in df.columns:
    print(df[col].value_counts())
"""1.passer les données en dataframe
2.inspecter les données
3.supprimer les doublons
4. renommer les colonnes pour une meilleures lecture
4.1 renomer la colonne "Sex " en supprimant l'espace --> OK
5. donner le bon type aux données (int, float, date, string, boolean)
6. on peut fusionner dans la colonne Type les données "boat" et "boating" -->OK
7. dans la colonne Country : mettre tous les pays en Majuscule, corriger le nom "ST. MAARTIN" en "ST. MARTIN", 
    supprimer la ligne "Between PORTUGAL & INDIA" et trouver d'autres info du genre
8. on peut supprimer les carctères spéciaux (les accents)
9. pour la colonne 'Sex', regarger pourquoi il y a 2 M, pour les 3 autres on les remplace par du non défini. --> OK
10. pour les ages, on peut créér une colonne qui sont des tranches d'age
11. pour 'Fatal', nettoyer les différents 'N' ajouter ce qui n'est pas connus par 'UNKNOWN' et 'F' par 'Y'-->OK
12. supprimer une des colonnes "href" car doublon. Si espace dans l'url ajouter un '-'

"""

2009.12.18        2
1983.06.15        2
1962.06.11.b      2
1915.07.06.a.R    2
1923.00.00.a      2
2012.09.02.b      2
1920.00.00.b      2
1907.10.16.R      2
2013.10.05        2
1913.08.27.R      2
2005.04.06        2
1980.07.00        2
1990.05.10        2
2006.09.02        2
1966.12.26        2
2014.08.02        2
1973.01.09        1
2003.02.15.b      1
2000.10.02        1
1894.06.15.b.R    1
2006.12.18        1
1962.01.11.b      1
2012.08.31        1
1994.05.15        1
2005.08.21        1
2015.11.03        1
1900.09.13        1
2000.05.07.b      1
1908.08.28.R      1
2004.02.26        1
                 ..
1966.09.27        1
2016.09.01        1
2015.10.08        1
2006.06.05        1
2011.09.11.a      1
2006.01.07        1
2001.07.25        1
1984.11.04        1
2006.09.18.R      1
2012.10.19        1
1913.11.21        1
1970.12.17        1
1942.00.00.f      1
1982.09.25        1
1877.12.15        1
1990.09.05        1
2006.07.28        1
2009.01.26.R      1
1941.00.00.e      1


'1.passer les données en dataframe\n2.inspecter les données\n3.supprimer les doublons\n4. renommer les colonnes pour une meilleures lecture\n4.1 renomer la colonne "Sex " en supprimant l\'espace --> OK\n5. donner le bon type aux données (int, float, date, string, boolean)\n6. on peut fusionner dans la colonne Type les données "boat" et "boating" -->OK\n7. dans la colonne Country : mettre tous les pays en Majuscule, corriger le nom "ST. MAARTIN" en "ST. MARTIN", \n    supprimer la ligne "Between PORTUGAL & INDIA" et trouver d\'autres info du genre\n8. on peut supprimer les carctères spéciaux (les accents)\n9. pour la colonne \'Sex\', regarger pourquoi il y a 2 M, pour les 3 autres on les remplace par du non défini. --> OK\n10. pour les ages, on peut créér une colonne qui sont des tranches d\'age\n11. pour \'Fatal\', nettoyer les différents \'N\' ajouter ce qui n\'est pas connus par \'UNKNOWN\' et \'F\' par \'Y\'-->OK\n12. supprimer une des colonnes "href" car doublon. Si espace dans l\'

In [7]:
#en voulant travailler sur la colonne "Sex", ça indiquait "undefined" --> il y a un problème de syntaxe. il faut donc renommer la colonne pour supprimer l'espace en trop.
#pareil pour la colonne "Species"
df =df.rename(columns={'Sex ':'Sex'})
df =df.rename(columns={'Species ':'Species'})
df.head()
#pour Vérification ==> df.loc[df['Sex'] == 'F']

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [8]:
df['Sex'].value_counts() 

M      4835
F       585
M         2
N         1
lli       1
.         1
Name: Sex, dtype: int64

In [9]:
#pour la colonne 'Sex', je corrige pour conserver 3 infos : M, F ou UNKNOWN
df['Sex'] =df['Sex'].replace(to_replace =["lli", "N", "."], value ="UNKNOWN") 
df['Sex'] =df['Sex'].replace(to_replace =["M "], value ="M")

"""
--- Autre solution ---
df.loc[df['Sex'] == 'lli']
df.at[1400,'Sex']='UNKNOWN'
df.loc[df['Sex'] == 'N']
df.at[4708,'Sex']='UNKNOWN'
df.loc[df['Sex'] == '.']
df.at[5201,'Sex']='UNKNOWN'
df.loc[df['Sex'] == 'M ']
df.at[341,'Sex']='M'
df.at[1363,'Sex']='M
'"""
#pour Vérification ==> df.loc[df['Sex'] == 'UNKNOWN']
df['Sex'].value_counts() 

M          4837
F           585
UNKNOWN       3
Name: Sex, dtype: int64

In [10]:
df['Fatal (Y/N)'].value_counts() 

N          4315
Y          1552
UNKNOWN      94
 N            8
n             1
F             1
#VALUE!       1
N             1
Name: Fatal (Y/N), dtype: int64

In [11]:
#ne garder que 3 info pour la colonne 'Fatal (Y/N)'
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =[" N", "n", "N "], value ="N") 
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =["F"], value ="Y")
df['Fatal (Y/N)'] =df['Fatal (Y/N)'].replace(to_replace =["#VALUE!"], value ="UNKNOWN")
df['Fatal (Y/N)'].value_counts() 

N          4325
Y          1553
UNKNOWN      95
Name: Fatal (Y/N), dtype: int64

In [12]:
"""from datetime import datetime

# Complete the call to convert the date column
df['Date'].value_counts()
df['Date'] = pd.to_datetime(df['Date'],errors='ignore')

# Confirm the date column is in datetime format

"""

"from datetime import datetime\n\n# Complete the call to convert the date column\ndf['Date'].value_counts()\ndf['Date'] = pd.to_datetime(df['Date'],errors='ignore')\n\n# Confirm the date column is in datetime format\n\n"

In [13]:
# Analyse de la colonne 'Date'
df['Date'].value_counts()

1957                    11
1942                     9
1956                     8
1958                     7
1941                     7
1950                     7
No date                  6
1949                     6
1955                     5
1940                     5
28-Jul-95                5
1959                     5
05-Oct-03                5
Oct-60                   5
Aug-56                   5
1954                     5
No date, Before 1963     5
12-Apr-01                5
1970s                    5
1876                     4
23-Jan-70                4
1890                     4
14-Jun-12                4
1995                     4
27-Dec-08                4
1898                     4
Before 1958              4
27-Jul-52                4
20-Sep-15                4
Before 1906              4
                        ..
Reported 03-Dec-2010     1
03-May-39                1
07-Aug-07                1
19-Oct-09                1
Reported 18-Oct-1907     1
16-Jul-64                1
O

In [14]:
df[df['Date'].str.match('No date')]
#supprimer les lignes 5907,5908,5965,5969,5970,5971 pour 'date'

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5899,ND.0110,"No date, late 1960s",0,Unprovoked,VENEZUELA,Los Roques Islands,NaN,Spearfishing,4 French divers,M,...,said to involve 2.5 m hammerhead sharks,http://waterco.com.br/ataque_tubarao.htm,ND-0110-FrenchDivers.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0110,ND.0110,94,NaN,NaN
5905,ND.0102,"No date, Before 1963",0,Unprovoked,BAHREIN,NaN,NaN,Pearl diving,male,M,...,Tiger shark,A.C. Doyle,ND-0102-Bahrein.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0102,ND.0102,88,NaN,NaN
5907,ND.0097,No date,0,Unprovoked,USA,Florida,"Key West, Monroe County",Kitesurfing,Paul Menta,M,...,NaN,Internet,ND-0097-PaulMenta.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0097,ND.0097,86,NaN,NaN
5908,ND.0096,No date,0,Unprovoked,REUNION,Grand'Anse,Petite-île,yachtsman in a zodiac,NaN,M,...,NaN,G. Van Grevelynghe,ND-0096-Zodiac-Reunion.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0096,ND.0096,85,NaN,NaN
5910,ND.0094,"No date, Before May-1996",0,Unprovoked,KOREA,South Korea,Cheju Island,Diving,"female, a Hae Nyeo",F,...,NaN,"K. Amsler, Divernet.com",ND-0094-HaeNyeo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0094,ND.0094,83,NaN,NaN
5911,ND.0093,"No date, Before Mar-1995",0,Unprovoked,FRENCH POLYNESIA,Tuamotus,Rangiroa,Fishing,male,M,...,NaN,J. Windh,ND-0093-Rangiroa.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0093,ND.0093,82,NaN,NaN
5913,ND.0090,"No date, Before Aug-1989",0,Unprovoked,VANUATU,Malampa Province,Malakula,NaN,female,F,...,NaN,S. Combs,ND-0090-Vanuatu-female.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0090,ND.0090,80,NaN,NaN
5914,ND.0089,"No date, Before Aug-1987",0,Provoked,VANUATU,Malampa Province,"Hokai, Malakula",Attempting to drive shark from area,a chief,M,...,A large hammerhead shark,S. Combs,ND-0089-VanuatuChief.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0089,ND.0089,79,NaN,NaN
5915,ND.0088,"No date, Before 1987",0,Unprovoked,IRAN,Khuzestan Province,"Ahvaz, on the Karun River",NaN,Mr. Jabar-Kaaby,M,...,Bull shark,B. Coad & F. Papahn,ND-0088-Jabar-Kaaby.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0088,ND.0088,78,NaN,NaN
5916,ND.0087,"No date, Before 1975",0,Provoked,USA,Florida,"Riviera Beach, Palm Beach County",Skin diving. Grabbed shark's tail; shark turne...,Carl Bruster,M,...,"Nurse shark, 2.1 m [7']","R. Skocik, p.176",ND-0087-Carl-Bruster.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0087,ND.0087,77,NaN,NaN


In [15]:
pat= r'Reported '
pat1= r'[1-9]+-[1-9]+'
pat2 = r'No date, Before *|^Before*|^No date,|No date -\d+' 
pat3 = r'No date -\d+' 
pat4 = r'No date,\sBefore\s\d+'



df['Date'] =df['Date'].replace(regex =[pat,pat1,pat4], value ="No date") 
df['Date'] =df['Date'].replace(regex =[pat3,pat2], value ="") 

#df['Date'] =df['Date'].replace(regex =r'No date[0-9]', value ="") 


df['Date'][5927]

df[df['Date'].str.match('No date')].head()


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
33,2016.07.14.4,No date 14-Jul-2016,2016,Unprovoked,BAHAMAS,NaN,Tiger Beach,Scuba Diving,Michael Dornellas,M,...,"Lemon shark, 9'","GrindTV, 7/14/2016",2016.07.14.R-TigerBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.14.R,2016.07.14.4,5960,NaN,NaN
34,2016.07.08.R,No date08-Jul-2016,2016,Unprovoked,SPAIN,Canary Islands,"Las Teresitas, Tenerife",Wading,female,F,...,Angel shark,La Opinion de Tenerife,2016.07.08.R-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.08.R,2016.07.08.R,5959,NaN,NaN
85,2016.03.03.R,No date03-Mar-2016,2016,Unprovoked,AUSTRALIA,South Australia,Wrights Bay,Fishing,Lee Taplin,M,...,Bronze whaler,"9 News, 3/1/2016",2016.03.03.R-Taplin.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.03.R,2016.03.03.R,5908,NaN,NaN
90,2016.02.10.R,No date10-Feb-2016,2016,Invalid,CAYMAN ISLANDS,Grand Cayman,Stingray City Bar,Feeding stingrays?,Richard Branson,M,...,No shark involvement,R. Branson,2016.02.10.R-Branson-stingray.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.02.10.R,2016.02.10.R,5903,NaN,NaN
100,2016.01.11.R,No date11-Jan-2016,2016,Unprovoked,AUSTRALIA,Queensland,"Happy Valley Beach, Caloundra",Surfing,Shane Hilder,M,...,Wobbegong shark,"ABC Sunshine Coast, 1/11/2016",2016.01.11.R-Hilder.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.01.11.R,2016.01.11.R,5893,NaN,NaN


In [16]:
df['Date'].value_counts()
#df[df['Date'].str.match('No date')].head()

No date                27
1957                   11
1942                    9
1956                    8
1958                    7
1950                    7
1941                    7
1949                    6
1955                    5
28-Jul-95               5
Aug-56                  5
Oct-60                  5
1970s                   5
1940                    5
1959                    5
1954                    5
05-Oct-03               5
12-Apr-01               5
27-Dec-08               4
20-Sep-15               4
1938                    4
1960                    4
09-Jul-94               4
1995                    4
1960s                   4
 1958                   4
 1906                   4
23-Jan-70               4
1945                    4
28-Dec-14               4
                       ..
No date02-Jul-1954      1
04-May-44               1
05-Oct-86               1
06-Jan-29               1
23-Feb-36               1
19-Jul-75               1
No date18-Mar-1999      1
25-May-15   

In [17]:
#df['Date'][5917]

In [18]:
#pour la colonne 'Type', je fusionne Boat et Boating
df['Type'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: Type, dtype: int64

In [19]:
df['Type'] =df['Type'].replace(to_replace =["Boating"], value ="Boat")
df['Type'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Boat             310
Sea Disaster     220
Name: Type, dtype: int64

In [20]:
#dans la colonne 'Country' : mettre tous les pays en Majuscule, corriger le nom "ST. MAARTIN" en "ST. MARTIN", 
    #remplacer les NaN par "NC"
    #mettre les noms en majuscule
    #passer en "NC" les lignes "Between PORTUGAL & INDIA" ou avec un "?" ou avec "/"
    #supprimer les espaces avant un mot
    
df['Country'].value_counts()

USA                         2116
AUSTRALIA                   1279
SOUTH AFRICA                 565
PAPUA NEW GUINEA             133
NEW ZEALAND                  125
BRAZIL                       102
BAHAMAS                       98
MEXICO                        81
ITALY                         71
FIJI                          62
PHILIPPINES                   59
REUNION                       57
NEW CALEDONIA                 51
MOZAMBIQUE                    44
CUBA                          42
SPAIN                         40
INDIA                         37
EGYPT                         36
CROATIA                       34
JAPAN                         32
PANAMA                        32
IRAN                          29
SOLOMON ISLANDS               29
GREECE                        25
HONG KONG                     24
JAMAICA                       23
FRENCH POLYNESIA              22
INDONESIA                     20
ENGLAND                       19
PACIFIC OCEAN                 17
          

In [21]:
#passer toutes les données en majuscules
df['Country'] = df['Country'].str.upper()
#identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'
    #df['Country'].isna()
    #df.loc[df['Country'].isna()]
df['Country']=df['Country'].fillna(value='NC')

df['Country'] =df['Country'].replace(to_replace =["ST. MAARTIN"], value ="ST. MARTIN")

#supprimer le ? de trop
df['Country'] = df['Country'].str.replace('[?]','')
    #df['Country']
    #df[df['Country'].str.match('NC')].head()

#remplacer les emplacements incertains par "NC" (qui ont "beetween", "/" dans l'intitulé)
    #df.loc[df['Country'] == 'NC']
pat6 = df.loc[df['Country'].str.contains(r'( / )')]
df['Country'] =df['Country'].replace(regex =[pat6], value ="NC") 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [22]:
df['Country'].str.lstrip()
#supprimer l'espace au début 
#pat7 = df.loc[df['Country'].str.contains(r'(^/s)')]

df.columns = df.columns.str.lstrip()
#df.at[4409,'Country'] = df.at[4409,'Country'].lstrip()
df['Country'].value_counts()

USA                      2116
AUSTRALIA                1279
SOUTH AFRICA              565
PAPUA NEW GUINEA          133
NEW ZEALAND               125
BRAZIL                    102
BAHAMAS                    98
MEXICO                     81
ITALY                      71
FIJI                       65
PHILIPPINES                59
REUNION                    57
NEW CALEDONIA              51
NC                         50
MOZAMBIQUE                 44
CUBA                       42
SPAIN                      40
INDIA                      37
EGYPT                      36
CROATIA                    34
JAPAN                      32
PANAMA                     32
IRAN                       29
SOLOMON ISLANDS            29
GREECE                     25
HONG KONG                  24
JAMAICA                    23
FRENCH POLYNESIA           22
INDONESIA                  20
ENGLAND                    19
                         ... 
GEORGIA                     1
 PHILIPPINES                1
CEYLON (SR

In [23]:
pat7 = df.loc[df['Country'].str.contains(r'(^ )')]
pat7
df['Country'] =df['Country'].replace(regex =[pat7], value ="")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [24]:
df['Unnamed: 22'].value_counts()

stopped here    1
Name: Unnamed: 22, dtype: int64

In [25]:
df['Unnamed: 23'].value_counts()

Teramo             1
change filename    1
Name: Unnamed: 23, dtype: int64

In [26]:
#travail sur la colonne "pdf"
    #remplacer 'pf' par 'pdf'
pat8 = df.loc[df['pdf'].str.contains(r'pf+')]
df['pdf'] =df['pdf'].replace(regex =[pat8], value ="pdf")
df.loc[df['pdf'].str.contains(r'pdf+')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [27]:
#travail sur la colonne "href"
    #identification des NaN et remplacement des NaN par NC (Non Communiqué) car il est difficile après de faire des 'replace'
df['href'].isna()
df.loc[df['href'].isna()]
df['href']=df['href'].fillna(value='NC')
    #remplacer 'pf' par 'pdf'
pat9 = df.loc[df['href'].str.contains(r'pf+')]
df['href'] =df['href'].replace(regex =[pat9], value ="pdf")
df.loc[df['href'].str.contains(r'pdf+')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [28]:
#suppression de la colonne "href formula" car identique à la colonne "href"
df.drop(columns=['href formula'],axis=1)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,15h15,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,14h30,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,15h40,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,Late afternoon,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [42]:
#travail sur la colonne "Time"
    #remplacer les NaN
df['Time'].value_counts()
df.loc[df['Time'].isna()]
df['Time']=df['Time'].fillna(value='NC')
    #changer "h" par ":" pour ensuite convertir en type Datetime
        #pour plus tard  --> df['Time']= df['Time'].str.replace('h',':',regex=True)

# Create a new column called df.elderly where the value is yes
# if df.age is greater than 50 and no if not
#df['AM_PM'] = np.where(df['Time']>=50, 'AM', 'PM')
# Assign a new column to df called 'age' with a list of ages
df = df.assign(AM_PM="")
df['AM_PM']=df['Time']

df['Time'].value_counts()
df['AM_PM'].value_counts()
test=df.loc[df['AM_PM'].str.contains(r'[^0-9][a-z]+')]
test

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,AM_PM
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN,Late afternoon
10,2016.09.05.a,05-Sep-16,2016,Unprovoked,AUSTRALIA,Western Australia,Injidup,Surfing,Fraser Penman,M,...,"Perth Now, 9/5/2016",2016.09.05.a-Penman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.a,2016.09.05.a,5983,NaN,NaN,Late afternoon
28,2016.07.20,20-Jul-16,2016,Provoked,AUSTRALIA,Queensland,"20 k off The Spit, off the Gold Coast",Fishing,Scott van Burck,M,...,"Nine News, 7/20/2016",2016.07.20-Burck.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.20,2016.07.20,5965,NaN,NaN,After noon
53,2016.06.04,04-Jun-16,2016,Unprovoked,EGYPT,Suez,Ain Sokhna,Swimming,Omar Abdel Qader,M,...,"Ahram Online, 6/4/2016",2016.06.04-Qader.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.06.04,2016.06.04,5940,NaN,NaN,Morning
55,2016.06.02.a,02-Jun-16,2016,Unprovoked,NEW CALEDONIA,NaN,"Côte-Blanche, Nouméa",Kite surfing,Pierre de Rotalier,M,...,"Les Nouvelles Caledoniennes, 6/3/2016",2016.06.02.a-Pierre.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.06.02.a,2016.06.02.a,5938,NaN,NaN,Afternoon
62,2016.05.18,18-May-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Swimming,Mark Wilson,M,...,"News4Jax, 5/19/2016",2016.05.18-Wilson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.05.18,2016.05.18,5931,NaN,NaN,Morning
65,2016.05.02,02-May-16,2016,Provoked,NEW ZEALAND,North Island,Cormandel,Fishing,male,M,...,Radio New Zealand 5/3/2016,2016.05.02-NZ-pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.05.02,2016.05.02,5928,NaN,NaN,Afternoon
81,2016.03.13,13-Mar-16,2016,Invalid,USA,California,"Bolsa Chica State Park, Orange County",Surfing,unknown,NaN,...,"Orange County Register, 3/13/2016",2016.03.13-BolsaChicaSurfer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.13,2016.03.13,5912,NaN,NaN,Morning
83,2016.03.10,10-Mar-16,2016,Unprovoked,FIJI,Vanua Levu,NaN,Diving for beche-de-mer,Maika Tabua,M,...,"Fiji Sun, 3/12/2016",2016.03.10-Tabua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.10,2016.03.10,5910,NaN,NaN,Afternoon
84,2016.03.04,04-Mar-16,2016,Unprovoked,USA,Florida,"Ocean Reef Park, Singer Island, Palm Beach County",NaN,male,M,...,WPTV. 3/4/2016,2016.03.04-OCPark.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.04,2016.03.04,5909,NaN,NaN,Afternoon


In [30]:
#en checkant les valeurs uniques, on constate qu'il y a une info qui devrait renseigner la colonne "Fatal (Y/N)"
#je cherche donc l'index 
df.loc[df['Time'].str.contains('FATAL')]
#df.at[4776].value_counts()
#je cherche ensuite si la colonne suivante n'a pas l'info
#pour cela, j'ai réduit le tableau pour vérifier s'il n'y a pas de décalage 
df1 = df[['Sex','Age','Injury','Fatal (Y/N)','Time','Species','Investigator or Source','pdf']]
df1.loc[4776]
#mais en fait c'est ok. Donc je ne corrige que la colonne "Fatal" pour cet index
df.at[4776,'Fatal (Y/N)']="Y"

In [130]:
#df_time = df[['Time','AM_PM']]
"""pat11 = df.loc[df['AM_PM'].str.contains('afternoon')]
df['AM_PM'] =df['AM_PM'].replace([pat11], value ="PM")

pat= r'Reported '
pat1= r'[1-9]+-[1-9]+'
pat2 = r'No date, Before *|^Before*|^No date,|No date -\d+' 
pat3 = r'No date -\d+' 
pat4 = r'No date,\sBefore\s\d+'

df['Date'] =df['Date'].replace(regex =[pat,pat1,pat4], value ="No date")"""

df[(df['col_name'].str.contains(r'afternoon')) & (df['col_name'].str.contains(r'Midnight'))]

#pat11 = df.loc[df['Time'].str.contains(r'[^0-9][a-z]+')]
#df_time['AM_PM'] =df_time['AM_PM'].replace(regex =[pat11], value ="test")
#df.loc[df['Time'] == (r'[^0-9][a-z]+') , 'AM_PM'] = 'test'
#df.loc[df['Time'] == "Late afternoon", 'AM_PM'] = 'PM'
df.loc[df['AM_PM'].str.contains('PM')]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,AM_PM
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN,PM
10,2016.09.05.a,05-Sep-16,2016,Unprovoked,AUSTRALIA,Western Australia,Injidup,Surfing,Fraser Penman,M,...,"Perth Now, 9/5/2016",2016.09.05.a-Penman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.a,2016.09.05.a,5983,NaN,NaN,PM
55,2016.06.02.a,02-Jun-16,2016,Unprovoked,NEW CALEDONIA,NaN,"Côte-Blanche, Nouméa",Kite surfing,Pierre de Rotalier,M,...,"Les Nouvelles Caledoniennes, 6/3/2016",2016.06.02.a-Pierre.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.06.02.a,2016.06.02.a,5938,NaN,NaN,PM
65,2016.05.02,02-May-16,2016,Provoked,NEW ZEALAND,North Island,Cormandel,Fishing,male,M,...,Radio New Zealand 5/3/2016,2016.05.02-NZ-pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.05.02,2016.05.02,5928,NaN,NaN,PM
83,2016.03.10,10-Mar-16,2016,Unprovoked,FIJI,Vanua Levu,NaN,Diving for beche-de-mer,Maika Tabua,M,...,"Fiji Sun, 3/12/2016",2016.03.10-Tabua.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.10,2016.03.10,5910,NaN,NaN,PM
84,2016.03.04,04-Mar-16,2016,Unprovoked,USA,Florida,"Ocean Reef Park, Singer Island, Palm Beach County",NaN,male,M,...,WPTV. 3/4/2016,2016.03.04-OCPark.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.03.04,2016.03.04,5909,NaN,NaN,PM
93,2016.02.04,04-Feb-16,2016,Unprovoked,AUSTRALIA,New South Wales,Hams Beach,Windsurfing,Andrew Morris,M,...,"B. Myatt, GSAF",2016.02.04-Morris.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.02.04,2016.02.04,5900,NaN,NaN,PM
101,2016.01.05,05-Jan-16,2016,Unprovoked,AUSTRALIA,Queensland,Heron Island,Wading,Nicolas Davis,M,...,"Nine News, 1/5/2016",2016.01.05-Davis.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.01.05,2016.01.05,5892,NaN,NaN,PM
112,2015.11.16,16-Nov-15,2015,Unprovoked,USA,Florida,"Playalinda Beach, Brevard County",Surfing,Aaron Conti,M,...,WFTV. 11/17/2015,2015.11.16-Conti.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.11.16,2015.11.16,5880,NaN,NaN,PM
137,2015.09.26,26-Sep-15,2015,Unprovoked,AUSTRALIA,Queensland,"Russel Island, Frankland Group",Snorkeling,female,F,...,"The Cairns Post, 9/28/2015",2015.09.26-QLD.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.09.26,2015.09.26,5855,NaN,NaN,PM


In [ ]:
"""
Afternoon
Morning
Evening
Late afternoon
Shortly before 13h00
10h00 / 11h00
20h45 (Sunset)
Ship aban-doned at 03h10
Possibly same incident as 2000.08.21
Between 06h00 & 07h20
Daybreak
Just before dawn
Mid afternoon
"After dark"
07h00 - 08h00
Late morning
Midday.
Nightfall
"""